In [ ]:
pip install attention

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow
from tensorflow.keras.layers import Input, Flatten, Dropout, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
### from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical
# define undersample strategy
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from attention import Attention
import tensorflow
import timeit
import tracemalloc
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve

In [21]:
lr=0.3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import glob
file_name_Despress = glob.glob('Depresjon/condition/*.csv')
file_name_Normal = glob.glob('Depresjon/control/*.csv')

### Start reading file from hardisk

In [4]:

def readfile(file_name):
    ID = 0
    result = []
    for f in file_name:
        df = pd.read_csv(f)
        out = [x[11:] for x in df.timestamp]
        
        df.timestamp = out     
        df_Pivot = pd.pivot_table(df,index=["timestamp"],columns=["date"],fill_value=0)
        if (df_Pivot.shape[-1]>7):
            for j in range(7, df_Pivot.shape[-1]+1):
                result.append(df_Pivot.iloc[:, j-7:j])
    return np.array(result)
        

In [5]:
df_Depress = readfile(file_name_Despress)
df_Nomal = readfile(file_name_Normal)

In [6]:
print(df_Depress.shape)
print(df_Nomal.shape)

(267, 1440, 7)
(547, 1440, 7)


In [7]:
df_Final = np.concatenate((df_Depress, df_Nomal))

In [8]:
label =[]
for i in range(0, df_Depress.shape[0]+df_Nomal.shape[0]):
    if i <df_Depress.shape[0]:
        label.append(1)
    else:
        label.append(0)

In [9]:
df_Final.shape

(814, 1440, 7)

### Split into training and testing set

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_Final, label, test_size=0.2, random_state=42)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*7)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*7)

### Defining the conf_matrix

In [12]:
from sklearn.metrics import confusion_matrix
    
import math
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel()
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    Recall=  (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= (2*(tp)/(2*tp+fp+fn))
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc

In [13]:
# Defining the learning Rate function
def lr_scheduler(epoch, lr):
    if epoch % 50 == 0 and epoch:
        lr = lr - lr*0.1
        return lr
    return lr

In [26]:
# Defining the model and training

def Res_model(X, y, index, epoch):
    # define the keras model
    print(X.shape)
    print(y.shape)
    time_steps = X.shape[1]
    input_dim = 1
    print((time_steps, input_dim))


    model_input = Input(shape=(time_steps, input_dim))
    x = GRU(128, return_sequences=True)(model_input)
    x = Attention(64)(x)
    x = Dense(32, activation = "relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(1, activation = "linear")(x)

    model = Model(model_input, x)

    model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error'])
    model.fit(X, y, epochs=epoch, batch_size=200, verbose = 1)
    model.save('/content/drive/MyDrive/Colab Notebooks/Khanh/GBossting/model_'+str(index)+'.h5')
    return model.predict(X)

In [16]:
# Combine separate models into 1 stacked model
def model_Predict(X,n_members):
    X = X.reshape(X.shape[0],X.shape[1], X.shape[2], 1)
    result = np.array([0.5] * len(X))
    for i in range(0, n_members):
        model = load_model('/content/drive/MyDrive/Colab Notebooks/Khanh/GBossting/model_'+str(i)+".h5")
        RM_predicted = model.predict(X)
        result = result+lr*RM_predicted.reshape((len(X), ))
    return result

In [17]:
X_test.shape

(163, 10080)

In [18]:
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

In [19]:
X_test.shape

(163, 10080, 1)

# **Start fitting**

In [ ]:
max_auc = 0.9
for num_Interation in range(2, 20, 3):
    rsFile = "/content/drive/MyDrive/Colab Notebooks/Khanh/GBossting/Result_OverIteration_Bagging_Numeric/NumberOfBag_"+str(num_Interation)+".csv"
    f2=open(rsFile,"w")
    f2.write('num_Interation, epoch, Time_train, Train_Peak_RAM, Time_predict, Peak_Time_prediction, thres, tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc, auc\n')
    for epoch in (10, 50, 100, 500):
        
    
        print("working on num_Interation: " + str(num_Interation) + "_with_epoch_"+str(epoch))
        
        Res = []
        predicted_Base =  np.array([0.5] * len(X_train))
        predicted = predicted_Base

        

        
        
        start = timeit.default_timer()
        tracemalloc.start()

        ## Start training
        for i in range(0, num_Interation):
            Res = y_train-predicted
            Predicted_Res = Res_model(X_train,Res, i, epoch)
            New_predicted = predicted+ lr*Predicted_Res.reshape((Predicted_Res.shape[0],))
            predicted = New_predicted

        current, peak = tracemalloc.get_traced_memory()
        stop = timeit.default_timer()
        Time_train=stop - start
        Train_Peak_RAM = peak / 10**6
        
        start = timeit.default_timer()
        tracemalloc.start()
        
        predicted_test = model_Predict(X_test, num_Interation)
        
        stop = timeit.default_timer()
        Time_predict=stop - start
        current, peak = tracemalloc.get_traced_memory()
        Peak_Time_prediction = peak / 10**6
            
            

        auc=roc_auc_score(y_test, predicted_test)


        thres = predicted_test.min()
        while (thres <= predicted_test.max()):
            thres = thres+(predicted_test.max()/300)
            yhat = []
            for i in range(len(predicted_test)):
                if predicted_test[i]>thres:
                    yhat.append(1)
                else:
                    yhat.append(0)

            yhat = np.array(yhat)

            tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc = conf_matrix(y_test, yhat)
            #auc = roc_plot(y_test, yhat)
            f2.write(str(num_Interation)+", "+str(epoch)+", "+str(Time_train)+", "+str(Train_Peak_RAM)+", "+str(Time_predict)+", "+str(Peak_Time_prediction)+", "+str(thres)+", "+str(tn)+", "+str(fp)+", "+str(fn)+", "+str(tp)+", "+str(acc)+", "+str(pre)+", "+str(Recall)+", "+str(spec)+", "+str(F1score)+", "+str(mcc)+", "+str(auc)+"\n")


    f2.close()
    print('WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!')       

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Epoch 67/100
4/4 [==============================] - 3s 615ms/step - loss: 0.1040 - mean_absolute_error: 0.2531
Epoch 68/100
4/4 [==============================] - 3s 617ms/step - loss: 0.1050 - mean_absolute_error: 0.2528
Epoch 69/100
4/4 [==============================] - 3s 614ms/step - loss: 0.1041 - mean_absolute_error: 0.2530
Epoch 70/100
4/4 [==============================] - 3s 608ms/step - loss: 0.1037 - mean_absolute_error: 0.2505
Epoch 71/100
4/4 [==============================] - 3s 621ms/step - loss: 0.1033 - mean_absolute_error: 0.2529
Epoch 72/100
4/4 [==============================] - 3s 625ms/step - loss: 0.1047 - mean_absolute_error: 0.2529
Epoch 73/100
4/4 [==============================] - 3s 617ms/step - loss: 0.1043 - mean_absolute_error: 0.2540
Epoch 74/100
4/4 [==============================] - 3s 611ms/step - loss: 0.1034 - mean_absolute_error: 0.2518
Epoch 75/100
4/4 [==============================] - 3s 